In [33]:
import matplotlib.pyplot as plt 
import numpy as np
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder , MinMaxScaler
from tqdm.auto import tqdm
from helper_function import accuracy_fn

In [178]:
import pandas as pd
output_title = LabelEncoder()
df = pd.read_csv("milknew_CF.csv")
df["Grade"] = output_title.fit_transform(df["Grade"])
X = df[["pH","Temprature","Taste","Odor","Fat","Turbidity","Colour"]].values
y = df["Grade"].values

scaler = StandardScaler()
X = scaler.fit_transform(X) 

X_train, X_test , y_train , y_test = train_test_split(X, y,test_size=0.2, random_state=1234)



X_train = torch.from_numpy(X_train).type(torch.float)
y_train = torch.from_numpy(y_train).type(torch.long)
X_test = torch.from_numpy(X_test).type(torch.float)
y_test = torch.from_numpy(y_test).type(torch.long)


train_dataset = TensorDataset(X_train,y_train)
test_dataset = TensorDataset(X_test,y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [114]:
a , b = next(iter(train_loader))
a.shape , b.shape

(torch.Size([32, 7]), torch.Size([32]))

In [179]:
class Milk_Model(nn.Module):
    def __init__(self, input_features , 
                 output_features , 
                 hidden_units = 7 ):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_features ,hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units , output_features),

        )
    def forward(self,x):
        return self.network(x)


In [181]:
Model = Milk_Model(7,3)
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=Model.parameters(),lr=0.01)
Model

Milk_Model(
  (network): Sequential(
    (0): Linear(in_features=7, out_features=7, bias=True)
    (1): ReLU()
    (2): Linear(in_features=7, out_features=3, bias=True)
  )
)

In [182]:
epochs = 100
train_acc , train_loss = 0 , 0
for epoch in tqdm(range((epochs))):
    Model.train()  
    for X_train , y_train in train_loader:
        pred = Model(X_train)
        pred_prob = torch.softmax(pred,dim=1).argmax(dim=1)
        loss = loss_fcn(pred,y_train)
        train_loss += loss
        train_acc += accuracy_fn(y_train,pred_prob)
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        
    
    train_acc /= len(train_loader)
    train_loss /= len(train_loader)
    
    if (epoch % 10 ) == 0 :
        print(f"<------------------------->")
        print(f'Epoch {epoch}, Train Acc: {train_acc:.4f}',)
    

print(f"Epoch: {epoch + 1} |, loss : {loss:.4f} |, Acc: | {train_acc:.2f}% ")

  0%|          | 0/100 [00:00<?, ?it/s]

<------------------------->
Epoch 0, Train Acc: 30.4167
<------------------------->
Epoch 10, Train Acc: 31.5719
<------------------------->
Epoch 20, Train Acc: 31.5808
<------------------------->
Epoch 30, Train Acc: 31.1975
<------------------------->
Epoch 40, Train Acc: 31.7031
<------------------------->
Epoch 50, Train Acc: 31.0856
<------------------------->
Epoch 60, Train Acc: 31.3137
<------------------------->
Epoch 70, Train Acc: 31.7076
<------------------------->
Epoch 80, Train Acc: 31.4608
<------------------------->
Epoch 90, Train Acc: 31.4550
Epoch: 100 |, loss : 1.2536 |, Acc: | 31.04% 


In [158]:
y_train[:10] , pred_prob[:10]

(tensor([1, 0, 1, 1, 2, 2, 0, 1, 1, 2]),
 tensor([2, 0, 1, 1, 2, 2, 1, 1, 1, 2]))